# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = os.environ.get('BATCH_SIZE', '16')
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 8 max_iteration = 5 model_type = base *********
n              iter (since)      4 loss (since)       4 acc (since)   4 ema acc (since) 4 acc (dev) (since)      dt
1             0.000 (0.000)       0.871 (0.871)       0.750 (0.750)       0.750 (0.750)       0.000 (0.000)  7.44 s
2             0.000 (0.000)       0.796 (0.722)       0.781 (0.812)       0.781 (0.812)       0.000 (0.000)  12.7 s
4             0.000 (0.000)       0.883 (0.969)       0.750 (0.719)       0.781 (0.781)       0.000 (0.000)  24.7 s
8             0.000 (0.000)       0.864 (0.846)       0.734 (0.719)       0.742 (0.703)       0.000 (0.000)  47.8 s
16            0.000 (0.000)       0.823 (0.783)       0.742 (0.750)       0.754 (0.766)       0.000 (0.000)  1.58 m
32            0.000 (0.000)       0.827 (0.830)       0.732 (0.723)       0.744 (0.734)       0.000 (0.000)   3.1 m
64            0.000 (0.000)       0.780 (0.732)       0.748 (0.764)       0.750 (0.756)       0.000 (0.000)  6.26 m
128  

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    os.environ['BATCH_SIZE'] = os.environ.get('BATCH_SIZE', '32')
    os.environ['LEARN_BATCH_SIZE'] = os.environ.get('LEARN_BATCH_SIZE', '16')
    os.environ['PREDICT_INNER_BATCH_SIZE'] = os.environ.get('PREDICT_INNER_BATCH_SIZE', '128')
    os.environ['STEP1_ITER'] = os.environ.get('STEP1_ITER', '1_augment8')
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 1 max_iteration = 10 model_type = base *********
n              iter (since)      4 loss (since)       4 acc (since)   4 acc ema (since) 4 acc (dev) (since)      nsamps (since)      dt
1             0.000 (0.000)       0.177 (0.177)       0.875 (0.875)       0.875 (0.875)       0.000 (0.000)     185.406 (185.406)  5.62 m
2             0.000 (0.000)       0.426 (0.675)       0.875 (0.875)       0.875 (0.875)       0.000 (0.000)      94.391 (3.375)  7.61 m
4             0.000 (0.000)       0.435 (0.445)       0.852 (0.828)       0.859 (0.844)       0.000 (0.000)      51.555 (8.719)  11.9 m
8             0.000 (0.000)       0.447 (0.459)       0.840 (0.828)       0.852 (0.844)       0.000 (0.000)      29.586 (7.617)  20.4 m
16            0.000 (0.000)       0.406 (0.365)       0.865 (0.891)       0.861 (0.871)       0.000 (0.000)      18.402 (7.219)  37.4 m
32            0.000 (0.000)       0.438 (0.469)       0.853 (0.840)       0.853 (0.844)       0.000 (0.000)      1

# Step 3: Prepare Submission Files

In [ ]:
def prepare_submission_probensemble(*, nvoters, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedNewsCat import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    taskllm_model_id = f'User_keq{k}_t5base_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'raw_taskllm')
    t5.load_adapter(taskllm_model_id, 'ema_taskllm')
    rhat_model_id = f'User_keq{k}_t5base_step2_iter{step2_iter}'
    t5.load_adapter(rhat_model_id, 'raw_rhat')
    t5.load_adapter(rhat_model_id, 'ema_rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_suffix="taskllm", model_id=taskllm_model_id, choices=dev.choices)
    rewardpredictor = RewardPredictor(t5=t5, adapter_suffix="rhat", model_id=rhat_model_id)

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0]-1, 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        safek = min(k, scores.shape[0])
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=safek).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    def make_prior(profile):
        from math import log

        c = [1]*len(dev.choices)
        for v in profile:
            c[dev.choices.index(v['category'])] += 1
        n = sum(c)

        return [ log(cnt) - log(n) for cnt in c ]
    
    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} nvoters {nvoters} ***')

    devgolds, testgolds = [], []
    with ProgressPrinter(f'{k} acc (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                prior = [ make_prior(ex['profile']) for ex in examples ]
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['article'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                votingpriors = [ [q]*min(nvoters, b-a) for a, b, q in zip(splits, splits[1:], prior) ]
                predicts = torch.cat(inner_batch(func = lambda p, q: taskllm.predict(p, prior=torch.Tensor(q).to(device)),
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []), sum(votingpriors, [])),
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ predicts[a:b,:].logsumexp(dim=0, keepdim=True).argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                if isdev:
                    targets = [ dev.choices.index(label) for label in labels ]
                    targets = torch.Tensor(targets).long().to(guesses.device)
                    acc = (guesses == targets).float().mean().item()
                else:
                    acc = None

                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': dev.choices[guess] })

            printer.addobs(acc)

    for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
        with open(f'lamp2u_{wut}golds_t5base_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}_nvoters{nvoters}.json', 'w') as jsonfile:
            json.dump({ 'task': 'LaMP_2', 'golds': golds }, jsonfile)
            
for nvoters in [1, 3, 5]:
    prepare_submission_probensemble(k=4, step1_iter='1_augment8', step2_iter='5_augment1', nvoters=nvoters)

*** step1_iter: 1_augment8 step2_iter: 5_augment1 nvoters 1 ***
n       4 acc (dev) (since)      dt
1             0.625 (0.625)  15.8 s
2             0.688 (0.750)  29.2 s
4             0.781 (0.875)  51.3 s
8             0.828 (0.875)  1.65 m
16            0.859 (0.891)  3.23 m
32            0.859 (0.859)  6.68 m
64            0.836 (0.812)  13.6 m
128           0.839 (0.842)  27.9 m
256           0.840 (0.875)  1.01 h
*** step1_iter: 1_augment8 step2_iter: 5_augment1 nvoters 3 ***
n       4 acc (dev) (since)      dt
1             0.625 (0.625)  16.8 s
2             0.688 (0.750)  31.5 s
4             0.781 (0.875)  55.6 s
8             0.844 (0.906)  1.79 m
16            0.867 (0.891)  3.46 m
32            0.859 (0.852)  7.03 m
64            0.834 (0.809)  14.2 m
128           0.838 (0.842)  28.9 m
256           0.839 (0.875)  1.04 h
*** step1_iter: 1_augment8 step2_iter: 5_augment1 nvoters 5 ***
n       4 acc (dev) (since)      dt
1             0.625 (0.625)  17.3 s
2             0.